In [82]:
import numpy as np
import pandas as pd
from mlfinlab.online_portfolio_selection import *
#from mlfinlab.online_portfolio_selection.BAH import BAH


In [83]:
stock_prices = pd.read_csv('../tests/test_data/stock_prices.csv', parse_dates=True, index_col='Date')
stock_prices = stock_prices.dropna(axis=1)
asset_name = list(stock_prices.columns)

In [3]:
bah = BAH()
bah.allocate(asset_names=asset_name, asset_prices=stock_prices, resample_by='W')
bah.weights

In [16]:
idx = stock_prices.index
make_array = np.array(stock_prices)
make_array

array([[ 49.2733345 ,  35.38999939, 106.63999939, ...,  77.36000061,
        101.40000153, 130.63000488],
       [ 49.71666718,  35.29000092, 107.        , ...,  77.45999908,
        101.51999664, 130.74000549],
       [ 48.22333145,  34.59999847, 106.97000122, ...,  77.55000305,
        101.65000153, 128.16999817],
       ...,
       [ 34.02000046,  23.70999908, 116.44999695, ...,  84.19000244,
        105.94000244, 176.63999939],
       [ 34.36000061,  24.04999924, 116.66999817, ...,  84.30000305,
        106.08999634, 179.        ],
       [ 34.68999863,  24.12000084, 117.05999756, ...,  84.37000275,
        106.        , 179.30999756]])

In [33]:
temp = pd.DataFrame(make_array, index=idx)

In [42]:
temp = pd.DataFrame(0, index=np.arange(1), columns=asset_name)


In [102]:
rp = stock_prices.pct_change()
rp

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-03,0.008997,-0.002826,0.003376,0.003779,0.001662,0.002054,-0.000527,0.006281,0.018160,0.011824,...,-0.001426,0.002345,-0.006847,0.003623,0.002823,-0.000483,-0.001377,0.001293,0.001183,0.000842
2008-01-04,-0.030037,-0.019552,-0.000280,-0.025602,-0.022719,0.002619,-0.024525,-0.023720,-0.030202,-0.036549,...,0.007614,-0.028139,-0.019125,-0.024332,-0.025630,-0.024506,0.000212,0.001162,0.001281,-0.019657
2008-01-07,0.007327,0.000867,-0.000187,-0.006182,0.001045,0.001817,0.007299,0.002984,-0.013732,-0.003871,...,0.020779,0.011368,0.028971,-0.000411,-0.001824,-0.000849,0.004349,0.000258,0.000689,-0.000858
2008-01-08,-0.007754,-0.006930,0.000748,-0.002333,-0.005610,0.002154,-0.007783,-0.014025,-0.015415,-0.017487,...,0.000231,-0.016860,-0.014078,-0.007539,-0.004417,-0.016148,-0.001162,0.001031,0.000197,-0.017258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,-0.012864,-0.024266,0.002937,-0.005291,-0.019195,0.008645,-0.023288,-0.039808,-0.033681,-0.032046,...,0.012299,-0.020905,-0.003386,-0.026079,-0.009569,-0.017910,0.024967,0.005018,-0.000661,-0.013938
2016-06-28,0.029476,0.020506,0.003445,0.015071,0.026221,-0.000177,0.030388,0.042888,0.008545,0.027847,...,0.003723,0.025676,0.024089,0.032779,0.020624,0.018036,0.002084,0.001189,0.000945,0.015304
2016-06-29,0.025316,0.013681,-0.000601,0.013100,0.018330,-0.004065,0.017695,0.026045,0.016945,0.018964,...,0.002733,0.015547,0.018094,0.020340,0.013472,0.017028,-0.007673,-0.000356,0.000472,0.016224


In [104]:
rp

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-03,0.008997,-0.002826,0.003376,0.003779,0.001662,0.002054,-0.000527,0.006281,0.018160,0.011824,...,-0.001426,0.002345,-0.006847,0.003623,0.002823,-0.000483,-0.001377,0.001293,0.001183,0.000842
2008-01-04,-0.030037,-0.019552,-0.000280,-0.025602,-0.022719,0.002619,-0.024525,-0.023720,-0.030202,-0.036549,...,0.007614,-0.028139,-0.019125,-0.024332,-0.025630,-0.024506,0.000212,0.001162,0.001281,-0.019657
2008-01-07,0.007327,0.000867,-0.000187,-0.006182,0.001045,0.001817,0.007299,0.002984,-0.013732,-0.003871,...,0.020779,0.011368,0.028971,-0.000411,-0.001824,-0.000849,0.004349,0.000258,0.000689,-0.000858
2008-01-08,-0.007754,-0.006930,0.000748,-0.002333,-0.005610,0.002154,-0.007783,-0.014025,-0.015415,-0.017487,...,0.000231,-0.016860,-0.014078,-0.007539,-0.004417,-0.016148,-0.001162,0.001031,0.000197,-0.017258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,-0.012864,-0.024266,0.002937,-0.005291,-0.019195,0.008645,-0.023288,-0.039808,-0.033681,-0.032046,...,0.012299,-0.020905,-0.003386,-0.026079,-0.009569,-0.017910,0.024967,0.005018,-0.000661,-0.013938
2016-06-28,0.029476,0.020506,0.003445,0.015071,0.026221,-0.000177,0.030388,0.042888,0.008545,0.027847,...,0.003723,0.025676,0.024089,0.032779,0.020624,0.018036,0.002084,0.001189,0.000945,0.015304
2016-06-29,0.025316,0.013681,-0.000601,0.013100,0.018330,-0.004065,0.017695,0.026045,0.016945,0.018964,...,0.002733,0.015547,0.018094,0.020340,0.013472,0.017028,-0.007673,-0.000356,0.000472,0.016224


In [90]:
stock_prices.astype(float).pct_change()



,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-03,0.008997,-0.002826,0.003376,0.003779,0.001662,0.002054,-0.000527,0.006281,0.018160,0.011824,...,-0.001426,0.002345,-0.006847,0.003623,0.002823,-0.000483,-0.001377,0.001293,0.001183,0.000842
2008-01-04,-0.030037,-0.019552,-0.000280,-0.025602,-0.022719,0.002619,-0.024525,-0.023720,-0.030202,-0.036549,...,0.007614,-0.028139,-0.019125,-0.024332,-0.025630,-0.024506,0.000212,0.001162,0.001281,-0.019657
2008-01-07,0.007327,0.000867,-0.000187,-0.006182,0.001045,0.001817,0.007299,0.002984,-0.013732,-0.003871,...,0.020779,0.011368,0.028971,-0.000411,-0.001824,-0.000849,0.004349,0.000258,0.000689,-0.000858
2008-01-08,-0.007754,-0.006930,0.000748,-0.002333,-0.005610,0.002154,-0.007783,-0.014025,-0.015415,-0.017487,...,0.000231,-0.016860,-0.014078,-0.007539,-0.004417,-0.016148,-0.001162,0.001031,0.000197,-0.017258
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,-0.012864,-0.024266,0.002937,-0.005291,-0.019195,0.008645,-0.023288,-0.039808,-0.033681,-0.032046,...,0.012299,-0.020905,-0.003386,-0.026079,-0.009569,-0.017910,0.024967,0.005018,-0.000661,-0.013938
2016-06-28,0.029476,0.020506,0.003445,0.015071,0.026221,-0.000177,0.030388,0.042888,0.008545,0.027847,...,0.003723,0.025676,0.024089,0.032779,0.020624,0.018036,0.002084,0.001189,0.000945,0.015304
2016-06-29,0.025316,0.013681,-0.000601,0.013100,0.018330,-0.004065,0.017695,0.026045,0.016945,0.018964,...,0.002733,0.015547,0.018094,0.020340,0.013472,0.017028,-0.007673,-0.000356,0.000472,0.016224


In [103]:
np.ones(10) * 1/10

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [99]:
name = stock_prices.columns.values

In [110]:
diff = stock_prices.pct_change()
diff = diff.fillna(0)
diff += 1
diff

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2008-01-03,1.008997,0.997174,1.003376,1.003779,1.001662,1.002054,0.999473,1.006281,1.018160,1.011824,...,0.998574,1.002345,0.993153,1.003623,1.002823,0.999517,0.998623,1.001293,1.001183,1.000842
2008-01-04,0.969963,0.980448,0.999720,0.974398,0.977281,1.002619,0.975475,0.976280,0.969798,0.963451,...,1.007614,0.971861,0.980875,0.975668,0.974370,0.975494,1.000212,1.001162,1.001281,0.980343
2008-01-07,1.007327,1.000867,0.999813,0.993818,1.001045,1.001817,1.007299,1.002984,0.986268,0.996129,...,1.020779,1.011368,1.028971,0.999589,0.998176,0.999151,1.004349,1.000258,1.000689,0.999142
2008-01-08,0.992246,0.993070,1.000748,0.997667,0.994390,1.002154,0.992217,0.985975,0.984585,0.982513,...,1.000231,0.983140,0.985922,0.992461,0.995583,0.983852,0.998838,1.001031,1.000197,0.982742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,0.987136,0.975734,1.002937,0.994709,0.980805,1.008645,0.976712,0.960192,0.966319,0.967954,...,1.012299,0.979095,0.996614,0.973921,0.990431,0.982090,1.024967,1.005018,0.999339,0.986062
2016-06-28,1.029476,1.020506,1.003445,1.015071,1.026221,0.999823,1.030388,1.042888,1.008545,1.027847,...,1.003723,1.025676,1.024089,1.032779,1.020624,1.018036,1.002084,1.001189,1.000945,1.015304
2016-06-29,1.025316,1.013681,0.999399,1.013100,1.018330,0.995935,1.017695,1.026045,1.016945,1.018964,...,1.002733,1.015547,1.018094,1.020340,1.013472,1.017028,0.992327,0.999644,1.000472,1.016224


In [114]:
diff.cumprod(axis=0)

,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2008-01-03,1.008997,0.997174,1.003376,1.003779,1.001662,1.002054,0.999473,1.006281,1.018160,1.011824,...,0.998574,1.002345,0.993153,1.003623,1.002823,0.999517,0.998623,1.001293,1.001183,1.000842
2008-01-04,0.978690,0.977677,1.003095,0.978080,0.978906,1.004679,0.974961,0.982412,0.987409,0.974843,...,1.006177,0.974140,0.974159,0.979203,0.977121,0.975022,0.998834,1.002456,1.002465,0.981168
2008-01-07,0.985861,0.978525,1.002907,0.972033,0.979928,1.006505,0.982077,0.985343,0.973850,0.971069,...,1.027085,0.985214,1.002382,0.978800,0.975338,0.974195,1.003179,1.002715,1.003156,0.980326
2008-01-08,0.978217,0.971743,1.003657,0.969766,0.974431,1.008673,0.974433,0.971524,0.958838,0.954088,...,1.027322,0.968604,0.988270,0.971421,0.971030,0.958463,1.002013,1.003749,1.003353,0.963408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,0.654106,0.647641,1.088897,0.852608,0.672846,1.291567,0.563785,0.585846,1.076755,0.813082,...,1.212402,0.723033,0.578386,0.581242,0.799584,1.377217,1.474571,1.087384,1.043294,1.310572
2016-06-28,0.673387,0.660921,1.092648,0.865457,0.690488,1.291339,0.580917,0.610972,1.085956,0.835723,...,1.216916,0.741597,0.592319,0.600295,0.816075,1.402056,1.477644,1.088676,1.044280,1.330628
2016-06-29,0.690434,0.669963,1.091992,0.876795,0.703145,1.286089,0.591197,0.626884,1.104358,0.851572,...,1.220242,0.753127,0.603037,0.612505,0.827069,1.425930,1.466306,1.088289,1.044773,1.352216


In [100]:
pd.DataFrame(make_array, index=idx, columns=name)


,EEM,EWG,TIP,EWJ,EFA,IEF,EWQ,EWU,XLB,XLE,...,XLU,EPP,FXI,VGK,VPL,SPY,TLT,BND,CSJ,DIA
Date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,49.273335,35.389999,106.639999,52.919998,78.220001,87.629997,37.939999,47.759998,41.299999,79.500000,...,42.090000,51.173328,55.983330,74.529999,67.309998,144.929993,94.379997,77.360001,101.400002,130.630005
2008-01-03,49.716667,35.290001,107.000000,53.119999,78.349998,87.809998,37.919998,48.060001,42.049999,80.440002,...,42.029999,51.293331,55.599998,74.800003,67.500000,144.860001,94.250000,77.459999,101.519997,130.740005
2008-01-04,48.223331,34.599998,106.970001,51.759998,76.570000,88.040001,36.990002,46.919998,40.779999,77.500000,...,42.349998,49.849998,54.536671,72.980003,65.769997,141.309998,94.269997,77.550003,101.650002,128.169998
2008-01-07,48.576668,34.630001,106.949997,51.439999,76.650002,88.199997,37.259998,47.060001,40.220001,77.199997,...,43.230000,50.416672,56.116669,72.949997,65.650002,141.190002,94.680000,77.570000,101.720001,128.059998
2008-01-08,48.200001,34.389999,107.029999,51.320000,76.220001,88.389999,36.970001,46.400002,39.599998,75.849998,...,43.240002,49.566669,55.326672,72.400002,65.360001,138.910004,94.570000,77.650002,101.739998,125.849998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-27,32.230000,22.920000,116.120003,45.119999,52.630001,113.180000,21.389999,27.980000,44.470001,64.639999,...,51.029999,37.000000,32.380001,43.320000,53.820000,199.600006,139.169998,84.120003,105.790001,171.199997
2016-06-28,33.180000,23.389999,116.519997,45.799999,54.009998,113.160004,22.040001,29.180000,44.849998,66.440002,...,51.220001,37.950001,33.160000,44.740002,54.930000,203.199997,139.460007,84.220001,105.889999,173.820007
2016-06-29,34.020000,23.709999,116.449997,46.400002,55.000000,112.699997,22.430000,29.940001,45.610001,67.699997,...,51.360001,38.540001,33.759998,45.650002,55.669998,206.660004,138.389999,84.190002,105.940002,176.639999


In [91]:
a = np.zeros(2)
b = np.ones(2)

In [92]:
a



array([0., 0.])

In [93]:
b

array([1., 1.])

In [94]:
np.vstack((a,b))

array([[0., 0.],
       [1., 1.]])